# **Integrando Python com SQLAlchemy**

In [1]:
!pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import ForeignKey
from sqlalchemy import Float
from sqlalchemy import select
from sqlalchemy import func
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
from sqlalchemy.sql.lambdas import insp
from sqlalchemy.types import CHAR

In [3]:
Base = declarative_base()

In [4]:
class Cliente(Base):
  __tablename__ = "cliente"
  id = Column(Integer, primary_key = True)
  nome = Column(String)
  cpf = Column(String)
  endereco = Column(String)

  conta = relationship(
      "Conta", 
      back_populates = "cliente", # Especifica o atributo correspondente na classe relacionada
      cascade = "all, delete-orphan" # Quando um cliente é excluído, todas as suas contas relacionadas são excluídas também (all).
  )

  def __repr__(self):
    """
      Retorna uma representação em string do objeto, para fins de depuração e exibição de informações.
    """
    return f"Cliente: (Id = {self.id}, Nome = {self.nome}, CPF = {self.cpf}, Endereço = {self.endereco})"
  
class Conta(Base):
  __tablename__ = "conta"
  id = Column(Integer, primary_key = True)
  tipo = Column(String(10), nullable = False)
  numero = Column(String(10), nullable = False)
  agencia = Column(Integer, nullable = False)
  saldo = Column(Float, nullable = False)
  cliente_id = Column(Integer, ForeignKey("cliente.id"), nullable = False)

  cliente = relationship(
      "Cliente", 
      back_populates = "conta"
  )

  def __repr__(self):
    return f"Conta: (Id = {self.id}, Tipo = {self.tipo}, Número = {self.numero}, Agência = {self.agencia}, Saldo = {self.saldo})"


In [5]:
# Conexão
engine = create_engine("sqlite://")

In [6]:
# Criando as classes como tabelas no BDs
Base.metadata.create_all(engine)

In [7]:
# Imprimindo o esquema do BD
engine_inspector = inspect(engine)

print(engine_inspector.has_table("conta"))
print(engine_inspector.get_table_names())

True
['cliente', 'conta']


# **Persistência dos dados**

In [8]:
with Session(engine) as session:
  cliente_1 = Cliente(
      nome = 'Maria Flor',
      cpf = 12312312312,
      endereco = 'Rua dos Bobos, 0 - Minas Gerais/MG',
      conta = [Conta(tipo='Corrente', agencia='0001', numero=20232023, saldo=5000)]
  )

  cliente_2 = Cliente(
      nome = 'João Gomes',
      cpf = 12345678910,
      endereco = 'Rua Tupinambá, 1000 - Minas Gerais/MG',
      conta = [Conta(tipo='Poupança', agencia='0001', numero=75895641, saldo=20)]
  )

In [9]:
# Adicionando uma lista de objetos ao banco de dados em uma única transação
session.add_all([cliente_1, cliente_2])

In [10]:
# Confirmando as alterações feitas aos objetos gerenciados pela sessão p/ enviar essas alterações para o banco de dados.
session.commit()

# **Recuperando informações do BD**

In [11]:
stmt = select(Cliente).where(Cliente.nome.in_(['Maria Flor', 'João Gomes']))
for cliente in session.scalars(stmt):
  print(cliente)

Cliente: (Id = 1, Nome = Maria Flor, CPF = 12312312312, Endereço = Rua dos Bobos, 0 - Minas Gerais/MG)
Cliente: (Id = 2, Nome = João Gomes, CPF = 12345678910, Endereço = Rua Tupinambá, 1000 - Minas Gerais/MG)


In [12]:
conta_stmt = select(Conta).where(Conta.cliente_id.in_([2]))
for conta in session.scalars(conta_stmt):
  print(conta)

Conta: (Id = 2, Tipo = Poupança, Número = 75895641, Agência = 1, Saldo = 20.0)


Recuperando dados de forma classificada

In [13]:
print(select(Cliente).order_by(Cliente.nome.desc()))

SELECT cliente.id, cliente.nome, cliente.cpf, cliente.endereco 
FROM cliente ORDER BY cliente.nome DESC


In [14]:
order_stmt = select(Cliente).order_by(Cliente.cpf.desc())
for i, result in enumerate (session.scalars(order_stmt)):
  print(f' Saída {i}: {result}')

 Saída 0: Cliente: (Id = 2, Nome = João Gomes, CPF = 12345678910, Endereço = Rua Tupinambá, 1000 - Minas Gerais/MG)
 Saída 1: Cliente: (Id = 1, Nome = Maria Flor, CPF = 12312312312, Endereço = Rua dos Bobos, 0 - Minas Gerais/MG)


In [15]:
order_stmt = select(Cliente).order_by(Cliente.endereco.desc())
for i in (session.scalars(order_stmt)):
  print(i)

Cliente: (Id = 1, Nome = Maria Flor, CPF = 12312312312, Endereço = Rua dos Bobos, 0 - Minas Gerais/MG)
Cliente: (Id = 2, Nome = João Gomes, CPF = 12345678910, Endereço = Rua Tupinambá, 1000 - Minas Gerais/MG)


Join Statment

In [16]:
print(select(Cliente.nome, Conta.tipo).join_from(Cliente, Conta))

SELECT cliente.nome, conta.tipo 
FROM cliente JOIN conta ON cliente.id = conta.cliente_id


In [17]:
join_stmt = select(Cliente.nome, Conta.saldo).join_from(Cliente, Conta)

for result in session.scalars(join_stmt):
  print (result)

Maria Flor
João Gomes


Fetch all

In [19]:
connection = engine.connect()
results = connection.execute(join_stmt).fetchall()
print('Executando statement a partir da connection. Nome e Saldo\n')
for result in results:
  print(result)


Executando statement a partir da connection. Nome e Saldo

('Maria Flor', 5000.0)
('João Gomes', 20.0)
